In [1]:
import pandas as pd
import numpy as np
from connection import connect
from sqlalchemy import text

from utils.model_loader import ModelRegistry
from utils.translate_language import convert_language

/nix/store/ywl7ldghn2an44lxxaxlnb3hg8w9yxsl-python3-3.13.7-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
co_oltp,etl_conn,_=connect()

# Extract

In [3]:
# Extraccion de datos desde el OLTP
# Se basa en humanresources.employee + person.person + department
query_employee = text("""
-- CTE para obtener el último salario.
WITH
ranked_pay_history AS (
    SELECT
        business_entity_id,
        rate_change_date,
        rate AS base_rate,
        pay_frequency,
        ROW_NUMBER() OVER(
            PARTITION BY business_entity_id
            ORDER BY rate_change_date DESC
        ) as rn
    FROM hr.employee_pay_history
),
-- CTE para obtener solo el departamento más reciente de cada empleado.
ranked_department_history AS (
    SELECT
        business_entity_id,
        department_id,
        start_date,
        end_date,
        ROW_NUMBER() OVER(
            PARTITION BY business_entity_id
            ORDER BY start_date DESC -- Ordenamos por la fecha de inicio más reciente
        ) as rn
    FROM hr.employee_department_history
)
SELECT
    e.business_entity_id AS employee_alternate_key,
    e.national_idnumber AS employee_national_id_alternate_key,
    p.first_name,
    p.middle_name,
    p.last_name,
    p.suffix,
    e.gender,
    e.marital_status,
    e.birth_date,
    e.hire_date,
    e.salaried_flag,
    e.vacation_hours,
    e.sick_leave_hours,
    e.current_flag,
    e.organization_level,
    e.job_title AS title,
    e.login_id,
    ea.email_address,
    pp.phone_number AS phone,
    d.name AS department_name,
    h.start_date,
    h.end_date,
    sp.territory_id AS sales_territory_alternate_key,
    rph.pay_frequency AS pay_frequency,
    rph.base_rate,
    NULL AS employee_photo
FROM hr.employee AS e
INNER JOIN person.person AS p
    ON e.business_entity_id = p.business_entity_id
LEFT JOIN person.email_address AS ea
    ON p.business_entity_id = ea.business_entity_id
LEFT JOIN person.person_phone AS pp
    ON p.business_entity_id = pp.business_entity_id
-- MODIFICAMOS EL JOIN: En lugar de unir directamente a la tabla de historial...
-- ...nos unimos a nuestro CTE que ya tiene los registros clasificados.
LEFT JOIN ranked_department_history AS h
    ON e.business_entity_id = h.business_entity_id
    AND h.rn = 1 -- ¡La condición clave! Solo queremos el registro más reciente (rn=1).
LEFT JOIN hr.department AS d
    ON h.department_id = d.department_id
LEFT JOIN sales.sales_person AS sp
    ON e.business_entity_id = sp.business_entity_id
LEFT JOIN ranked_pay_history AS rph
    ON e.business_entity_id = rph.business_entity_id
    AND rph.rn = 1
""")

df_emp = pd.read_sql(query_employee, co_oltp)
print(f"Registros extraidos: {len(df_emp)}")
print(df_emp.head(3))

Registros extraidos: 290
   employee_alternate_key employee_national_id_alternate_key first_name  \
0                       1                          295847284        Ken   
1                       2                          245797967      Terri   
2                       3                          509647174    Roberto   

  middle_name   last_name suffix gender marital_status  birth_date  \
0           J     Sánchez   None      M              S  1969-01-29   
1         Lee       Duffy   None      F              S  1971-08-01   
2        None  Tamburello   None      M              M  1974-11-12   

    hire_date  ...                  login_id                 email_address  \
0  2009-01-14  ...      adventure-works\ken0      ken0@adventure-works.com   
1  2008-01-31  ...    adventure-works\terri0    terri0@adventure-works.com   
2  2007-11-11  ...  adventure-works\roberto0  roberto0@adventure-works.com   

          phone  department_name  start_date end_date  \
0  697-555-0142        

# Transform

In [4]:
# Enlaza con DimSalesTerritory (territorio de ventas)
df_terr = pd.read_sql(
    text("SELECT sales_territory_key, sales_territory_alternate_key FROM dim_sales_territory;"),
    etl_conn
)

In [5]:
# Algunos empleados pueden tener territorio asignado si son vendedores
# (se usa la tabla sales.sales_person)
sales_person = pd.read_sql(
    text("SELECT business_entity_id AS employee_alternate_key, territory_id FROM sales.sales_person;"),
    co_oltp
)

df_emp = df_emp.merge(
    sales_person,
    on='employee_alternate_key',
    how='left'
)

df_emp = pd.merge(
    df_emp,
    df_terr,
    on='sales_territory_alternate_key',  # Usamos 'on' porque los nombres de columna coinciden
    how='left'
)

In [6]:
df_emp['name_style'] = 0
df_emp['sales_person_flag'] = df_emp['sales_territory_key'].notnull().astype(int)
df_emp['current_flag'] = df_emp['current_flag'].astype(int)
df_emp['salaried_flag'] = df_emp['salaried_flag'].astype(int)

In [7]:
# Limpia columnas textuales nulas
for col in ['department_name', 'title']:
    df_emp[col] = df_emp[col].fillna('Unknown')

In [8]:
df_emp['status'] = np.where(
        pd.isnull(df_emp['end_date']),
        'Current',
        None
)

In [9]:
# Selecciona las columnas finales
final_columns = [
    'employee_alternate_key',
    #'parent_employee_key',
    'employee_national_id_alternate_key',
    'sales_territory_key',
    'first_name',
    'last_name',
    'middle_name',
    'name_style',
    'title',
    'gender',
    'marital_status',
    'birth_date',
    'hire_date',
    'login_id',
    'email_address',
    'phone',
    #'emergency_contact_name',
    #'emergency_contact_phone',
    'salaried_flag',
    'vacation_hours',
    'sick_leave_hours',
    'current_flag',
    'sales_person_flag',
    'department_name',
    'pay_frequency',
    'base_rate',
    'start_date',
    'end_date',
    'status',
    'employee_photo',
]

In [10]:
df_to_load = df_emp[final_columns]
print("Columnas finales:", df_to_load.columns.tolist())

Columnas finales: ['employee_alternate_key', 'employee_national_id_alternate_key', 'sales_territory_key', 'first_name', 'last_name', 'middle_name', 'name_style', 'title', 'gender', 'marital_status', 'birth_date', 'hire_date', 'login_id', 'email_address', 'phone', 'salaried_flag', 'vacation_hours', 'sick_leave_hours', 'current_flag', 'sales_person_flag', 'department_name', 'pay_frequency', 'base_rate', 'start_date', 'end_date', 'status', 'employee_photo']


In [11]:
df_to_load

,employee_alternate_key,employee_national_id_alternate_key,sales_territory_key,first_name,last_name,middle_name,name_style,title,gender,marital_status,...,sick_leave_hours,current_flag,sales_person_flag,department_name,pay_frequency,base_rate,start_date,end_date,status,employee_photo
0,1,295847284,NaN,Ken,Sánchez,J,0,Chief Executive Officer,M,S,...,69,1,0,Executive,2,125.5000,2009-01-14,None,Current,None
1,2,245797967,NaN,Terri,Duffy,Lee,0,Vice President of Engineering,F,S,...,20,1,0,Engineering,2,63.4615,2008-01-31,None,Current,None
2,3,509647174,NaN,Roberto,Tamburello,None,0,Engineering Manager,M,M,...,21,1,0,Engineering,2,43.2692,2007-11-11,None,Current,None
3,4,112457891,NaN,Rob,Walters,None,0,Senior Tool Designer,M,S,...,80,1,0,Tool Design,2,29.8462,2010-05-31,None,Current,None
4,5,695256908,NaN,Gail,Erickson,A,0,Design Engineer,F,M,...,22,1,0,Engineering,2,32.6923,2008-01-06,None,Current,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,286,758596752,9.0,Lynn,Tsoflias,N,0,Sales Representative,F,S,...,38,1,1,Sales,2,23.0769,2013-05-30,None,Current,None
286,287,982310417,NaN,Amy,Alberts,E,0,European Sales Manager,F,M,...,30,1,0,Sales,2,48.1010,2012-04-16,None,Current,None
287,288,954276278,8.0,Rachel,Valdez,B,0,Sales Representative,F,S,...,37,1,1,Sales,2,23.0769,2013-05-30,None,Current,None
288,289,668991357,10.0,Jae,Pak,B,0,Sales Representative,F,M,...,38,1,1,Sales,2,23.0769,2012-05-30,None,Current,None


# Load

In [13]:
# Carga al DW
df_to_load.to_sql(
    'dim_employee',
    etl_conn,
    schema='dw',
    if_exists='append',
    index=False
)

290